##                               Homework 11: ISYE6501
##                               Author: Nilesh Domadia
##                                      Date: 11/3/2020

Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls. 


1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)

In [58]:
# ---------- Import modules -----------

In [59]:
from pulp import *
import pandas as pd
import numpy as np

In [60]:
# ------------ Read data into the dataframe ---------------

In [61]:
df = pd.read_excel('E:\\georgiatech\\assignment\\HW11\\data\\diet.xls')
# lets look at the data
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [62]:
# ------------ rows 0:64 (Excel calls them 1-65) is the food data table lets extract them into a different dataframe.

In [63]:
diet = df[0:64]

In [64]:
# ------------ Extract the bottom constraints

In [65]:
diet_constraints = pd.read_excel('E:\\georgiatech\\assignment\\HW11\\data\\diet.xls', skiprows=66, header=None).iloc[:,2:]
diet_constraints.columns = df.columns[2:]

In [66]:
# ------------display Constraints------------ 

In [67]:
diet_constraints

,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Minimum daily intake,1500,30,20,800,130,125,60,1000,400,700,10
1,Maximum daily intake,2500,240,70,2000,450,250,100,10000,5000,1500,40


In [68]:
# ------------  create list for mins and maxes (all foods) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints

In [69]:
amin = [1500, 30, 20, 800, 130, 125, 60, 1000, 400, 700, 10]
amax = [2500, 240, 70, 2000, 450, 250, 100, 10000, 5000, 1500, 40]

In [70]:
data = diet.values.tolist()

In [71]:
# ------------ create master foods dictionary

In [72]:
foods = [x[0] for x in data]

In [73]:
#-------------------- append collection of contraints for each column 

In [74]:
cals = []
for j in range(0,11):
    cals.append(dict([(x[0], float(x[j+3])) for x in data]))    

In [75]:
#--------------------  define the cost dictionary

In [76]:
# define the cost dictionary
cost = dict([(x[0], float(x[1])) for x in data])

In [77]:
# ------------ Create a new LP Problem ------------ 
#
# This problem is a minimization problem (find the *lowest* cost), so "LpMinimize" is the second parameter.

In [78]:
problem1 = LpProblem('PuLPTutorial', LpMinimize)

In [79]:
# ------------ define the variables - continous

In [80]:
foodVars = LpVariable.dicts("foods", foods,0)

In [81]:
# ------------ define the variables - binary

In [82]:
chosenVars = LpVariable.dicts("Chosen",foods,0,1,"Binary")

In [83]:
# ------------  dictionary of lp variables 

In [84]:
x = LpVariable.dicts("x", foods, 0)

In [85]:
# ------------ define the OBJECTIVE function  ------------

In [86]:
problem1 += lpSum([cost[f] * foodVars[f] for f in foods])

In [87]:
# ------------  add CONSTRAINTS for all foods  ------------

In [88]:
# add constraints for all foods
for i in range(0,11):
    dot_B_x = pulp.lpSum([cals[i][j] * foodVars[j] for j in foods])
    condition1 = amin[i] <= + dot_B_x
    problem1 += condition1
    
for i in range(0,11):
    dot_B_x = pulp.lpSum([cals[i][j] * foodVars[j] for j in foods])
    condition2 = amax[i] >= + dot_B_x
    problem1 += condition2

In [89]:
# ------------  SOLVE  for optimal solution  ------------

In [90]:
problem1.solve()

1

In [91]:
# ------------  Status of the solution is Optimal ------------

In [92]:
LpStatus[problem1.status]

'Optimal'

In [93]:
# ------------  output solution to console values of the variable to the console ----------------

In [94]:
print(problem1.variables())

[foods_2%_Lowfat_Milk, foods_3.3%_Fat,Whole_Milk, foods_Apple,Raw,W_Skin, foods_Apple_Pie, foods_Bagels, foods_Banana, foods_Beanbacn_Soup,W_Watr, foods_Bologna,Turkey, foods_Butter,Regular, foods_Cap'N_Crunch, foods_Carrots,Raw, foods_Celery,_Raw, foods_Cheddar_Cheese, foods_Cheerios, foods_Chicknoodl_Soup, foods_Chocolate_Chip_Cookies, foods_Corn_Flks,_Kellogg'S, foods_Couscous, foods_Crm_Mshrm_Soup,W_Mlk, foods_Frankfurter,_Beef, foods_Frozen_Broccoli, foods_Frozen_Corn, foods_Grapes, foods_Ham,Sliced,Extralean, foods_Hamburger_W_Toppings, foods_Hotdog,_Plain, foods_Kielbasa,Prk, foods_Kiwifruit,Raw,Fresh, foods_Lettuce,Iceberg,Raw, foods_Macaroni,Ckd, foods_Malt_O_Meal,Choc, foods_New_E_Clamchwd,W_Mlk, foods_Neweng_Clamchwd, foods_Oatmeal, foods_Oatmeal_Cookies, foods_Oranges, foods_Peanut_Butter, foods_Peppers,_Sweet,_Raw, foods_Pizza_W_Pepperoni, foods_Poached_Eggs, foods_Popcorn,Air_Popped, foods_Pork, foods_Potato_Chips,Bbqflvr, foods_Potatoes,_Baked, foods_Pretzels, foods_Rais

In [95]:
# ------------   print the foods of the optimal diet and units ----------------

In [96]:
print('Optimization Solution:')
print('------------------------------')
for var in problem1.variables():
    if var.varValue > 0:
        if str(var).find('Chosen'):
            print(str(var.varValue) + "  units of  " + str(var))
            

Optimization Solution:
------------------------------
52.64371  units of  foods_Celery,_Raw
0.25960653  units of  foods_Frozen_Broccoli
63.988506  units of  foods_Lettuce,Iceberg,Raw
2.2929389  units of  foods_Oranges
0.14184397  units of  foods_Poached_Eggs
13.869322  units of  foods_Popcorn,Air_Popped


In [97]:
# ------------   print the COST of the Optimal Diet  ----------------       

In [98]:
print("Total cost of food = $%.2f" % value(problem1.objective))

Total cost of food = $4.34


**The cheapest diet that satisfies the maximum and minimum daily nutrition constraints will cost $4.34**

## Part 2

	
Please add to your model the following constraints (which might require adding more variables) and solve the new model:
      
      a.	If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and   how much is part of the diet. You’ll also need to write a constraint to link them.)
      
      b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
      
      c.	To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]


In [99]:
#----------- add contraints (a) amount greater than .1 or less than large amount - if chosen

In [100]:
for f in foods:
    problem1 += foodVars[f] <= 10000 * chosenVars[f]
    problem1 += foodVars[f] >= .1 * chosenVars[f]

In [101]:
#----------- add contraints (b) to eat at most one of a group of foods    

In [102]:
problem1 += chosenVars['Frozen Broccoli'] + chosenVars['Celery, Raw'] <= 1, 'At most one Broccoli / Celery'

In [103]:
# ------------ make a list of proteins

In [104]:
proteinList = [
       'Bologna,Turkey', 'Frankfurter, Beef','Ham,Sliced,Extralean',
       'Hamburger W/Toppings', 'Hotdog, Plain', 'Kielbasa,Prk',
       'Pizza W/Pepperoni', 'Poached Eggs',
       'Pork', 'Roasted Chicken', 'Sardines in Oil',
       'Scrambled Eggs',
       'Splt Pea&Hamsoup', 'Vegetbeef Soup',
       'White Tuna in Water']

In [105]:
#-----------  add contraint (c) at least 3 kinds of meat/poultry/fish/eggs must be selected. 

In [106]:
# Build constraints of at least 3
problem1 += lpSum([chosenVars[p] for p in proteinList]) >= 3

In [107]:
# ------------  SOLVE  for optimal solution  ------------

In [108]:
problem1.solve()

1

In [109]:
# ------------  Status of the solution is Optimal ------------

In [110]:
LpStatus[problem1.status]

'Optimal'

In [111]:
# ------------   print the foods of the optimal diet and units ----------------

In [112]:
print('Optimization Solution:')
print('------------------------------')

for var in problem1.variables():
    if var.varValue > 0:
        if str(var).find('Chosen'):
            print(str(var.varValue) + " units of " + str(var))
            

Optimization Solution:
------------------------------
42.399358 units of foods_Celery,_Raw
0.1 units of foods_Kielbasa,Prk
82.802586 units of foods_Lettuce,Iceberg,Raw
3.0771841 units of foods_Oranges
1.9429716 units of foods_Peanut_Butter
0.1 units of foods_Poached_Eggs
13.223294 units of foods_Popcorn,Air_Popped
0.1 units of foods_Scrambled_Eggs


**Notice that 3 protien foods Kielbasa,Prk; Poached_Eggs and Scrambled_Eggs were added**

**Also notice that Celery,_Raw at most one of the required group was also added**

In [113]:
# ------------   print the COST of the Optimal Diet  ----------------       

In [114]:
print("Total cost of food = $%.2f" % value(problem1.objective))

Total cost of food = $4.51


**The cheapest diet that satisfies the minimum of 1/10 serving, one of the dislike (Celery) and 3 proteins as constraints will cost $4.51**